In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

# HW2:  Модель Word2vec
### Выполнили:  

* Булгаков Дмитрий

* Тефикова Алие

### Группа ИАД-2

# 0. Loading data

Составьте достаточно большую коллекцию текстов на любом языке или ис-пользуйте тексты отсюда:<br>
https://www.kaggle.com/c/word2vec-nlp-tutorial/data.

In [2]:
text_dataset = pd.read_csv('data/unlabeledTrainData.csv', sep=';', encoding = 'latin1')

In [6]:
text_dataset

,id,review
0,9999_0,"Watching Time Chasers, it obvious that it was ..."
1,45057_0,I saw this film about 20 years ago and remembe...
2,15561_0,"Minor Spoilers<br /><br />In New York, Joan Ba..."
3,7161_0,I went to see this film with a great deal of e...
4,43971_0,"Yes, I agree with everyone on this site this m..."
5,36495_0,Jennifer Ehle was sparkling in \Pride and Prej...
6,49472_0,Amy Poehler is a terrific comedian on Saturday...
7,36693_0,A plane carrying employees of a large biotech ...
8,316_0,"A well made, gritty science fiction movie, it ..."
9,32454_0,Incredibly dumb and utterly predictable story ...


# 1. Fitting Word2Vec model

Обучите модель word2vec. Оцените время обучения модели,используя модуль time. Есть два варианта обучения модели: по отзывам целиком и с уче-там границ предложений. В принципе, погрешностью, которая возникает в первом случае можно пренебречь, но если вы хотите учитывать границы предложений, то можно использовать sent_tokenize из nltk.tokenize

In [4]:
from nltk.tokenize import sent_tokenize

In [30]:
#lens  = []
for i in text_dataset.review.index:
    sent_tokenize_list = sent_tokenize(text_dataset.review.values[i])
    #lens.append(len(sent_tokenize_list))

In [36]:
sent_tokenize_list

['This not-very-good mummy-alien flick does feature a cornucopia of your favorite movie stars like BEN MURPHY (\\Riding With Death\\"!',
 '), Bob Random (\\"Village of the Giants\\"!',
 '), Darwin Joston (Napolean in \\"Assault on Precinct 13\\"!',
 '), Austin Stoker (Bishop in \\"Assault on Precinct 13\\"!)',
 'as Dr. Ken Melrose, Nina Axelrod (\\"Motel Hell\\"), Shari Belafonte-Harper (who was a voice on \\"Rick Moranis in Gravedale High\\"!!!',
 '), Clint Young (\\"Rape Squad\\", \\"Switchblade Sisters\\"), and best of all, Pathmark pitchman JAMES KAREN (\\"Poltergiest\\", \\"Return of the Living Dead\\")!<br /><br />The plot is this: A priceless sarcophagus has been recovered in King Tut\'s tomb.',
 "Of course, it's taken to a small college in California where bumbling students can manhandle it and screw up X-raying it, instead of, oh, say, a museum.",
 'One loathsome, video-game loving student steals some diamond-type-thingees from the sarcophagus, sending the mummy on a poorly pl

In [50]:
%time
model = Word2Vec(sent_tokenize_list)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs
